<div align='center'>

# Reinforcement Learning (RL)

### Part of Scientific-ML-Notes 

[![GitHub](https://img.shields.io/badge/GitHub-Repository-black?logo=github&scale=5)](https://github.com/mhnaderi/Scientific-ML-Notes)

---

**Note: The content of this notebook is inspired by and adapted from the book *Understanding Deep Learning* by Simon J.D. Prince.**

Reinforcement Learning (RL) is a framework for sequential decision-making where an agent learns to interact with an environment to maximize cumulative rewards. For example, an RL algorithm might control the actions of a character (the agent) in a video game (the environment) with the goal of achieving the highest possible score (the reward).

Consider the task of learning to play chess. In this scenario, the agent receives a reward of $+1$, $-1$, or $0$ at the end of the game if it wins, loses, or draws, respectively, and receives $0$ at every other move. This setup illustrates several challenges inherent in RL:

- **Sparse Rewards**: The agent must complete an entire game before receiving any feedback, making learning more difficult.
- **Temporal Credit Assignment Problem**: Rewards are delayed and may result from actions taken many moves earlier. The agent needs to associate the eventual reward with the critical actions that led to it.
- **Stochastic Environment**: The opponent's moves are unpredictable and may vary even in identical situations, complicating the agent's ability to discern whether an action was truly effective or merely lucky.
- **Exploration-Exploitation Trade-Off**: The agent must balance exploring new strategies (e.g., trying different opening moves) with exploiting known successful strategies (e.g., sticking to a proven opening).

## Markov Decision Processes, Returns, and Policies

Reinforcement learning involves mapping observations from an environment to actions, with the goal of maximizing a cumulative numerical reward. Typically, we aim to learn a policy that maximizes the expected return within the framework of a Markov Decision Process (MDP).

### Markov Process

A Markov process models a system where the world is always in one of several possible states. The Markov property implies that the probability of transitioning to the next state depends solely on the current state, not on the sequence of preceding states. These transitions are governed by the probabilities $\Pr(s_{t+1} \mid s_t)$, representing the likelihood of moving to state $s_{t+1}$ given the current state $s_t$, where $t$ denotes the time step. Therefore, a Markov process generates a sequence of states $s_1, s_2, s_3, \ldots$ as it evolves over time.

### Markov Reward Process

A Markov reward process extends a Markov process by associating rewards with state transitions. It includes a distribution $\Pr(r_{t+1} \mid s_t)$ over possible rewards $r_{t+1}$ received at time $t+1$, given the current state $s_t$. This process produces a sequence of states and rewards: $s_1, r_2, s_2, r_3, s_3, r_4, \ldots$. Additionally, it incorporates a discount factor $\gamma \in (0, 1]$, which is used to calculate the return $G_t$ at time $t$:

$$
G_t = \sum_{k=0}^{\infty} \gamma^k r_{t+k+1}
$$

The return $G_t$ represents the cumulative discounted future rewards from time $t$ onward. A discount factor less than one places more importance on immediate rewards compared to those received in the distant future.

### Markov Decision Process (MDP)

A Markov Decision Process (MDP) further extends the Markov reward process by introducing a set of possible actions at each time step. The action $a_t$ taken by the agent influences the transition probabilities, which are now expressed as $\Pr(s_{t+1} \mid s_t, a_t)$. The rewards may also depend on the action, represented as $\Pr(r_{t+1} \mid s_t, a_t)$. An MDP generates a sequence of states, actions, and rewards: $s_1, a_1, r_2, s_2, a_2, r_3, s_3, a_3, r_4, \ldots$. In this framework, the entity making decisions and taking actions is called the agent.

### Partially Observable Markov Decision Process (POMDP)

In a Partially Observable Markov Decision Process (POMDP), the agent cannot directly observe the true state of the environment. Instead, it receives observations $o_t$ drawn from the distribution $\Pr(o_t \mid s_t)$. The POMDP produces a sequence of states, observations, actions, and rewards: $s_1, o_1, a_1, r_2, s_2, o_2, a_2, r_3, s_3, o_3, a_3, r_4, \ldots$. Generally, each observation provides partial information about the state, making it insufficient to uniquely identify the current state.

### Policy

A policy defines the behavior of the agent by specifying how it selects actions based on the current state. Policies can be deterministic, where a specific action is chosen for each state, or stochastic, where the policy defines a probability distribution over possible actions. A stochastic policy $\pi(a \mid s)$ gives the probability of taking action $a$ when in state $s$, from which actions are sampled. A deterministic policy can be viewed as a special case where the probability for a chosen action is one, and zero for all others. Policies can also be stationary (depending only on the current state) or non-stationary (also depending on the time step).

The interaction between the agent and the environment forms a loop. At each time step, the agent receives the current state $s_t$ and reward $r_t$ from the environment. Based on this information, the agent may update its policy $\pi(a_t \mid s_t)$ and selects the next action $a_t$. The environment then transitions to a new state $s_{t+1}$ according to the transition probabilities $\Pr(s_{t+1} \mid s_t, a_t)$ and provides a new reward $r_{t+1}$ based on $\Pr(r_{t+1} \mid s_t, a_t)$.

This interaction can be visualized as a continuous loop between the agent and the environment. The agent observes the current state and reward, updates its policy if necessary, and selects an action. The environment receives the action, transitions to a new state, and provides a new reward, which the agent then uses for the next decision.

## Expected Return

Previously, we introduced the concept of a Markov Decision Process (MDP) and discussed how an agent selects actions based on a policy. Our objective is to find a policy that maximizes the expected return. In this section, we will formalize this idea by assigning values to each state $s_t$ and state-action pair $(s_t, a_t)$.

### State and Action Values

The return $G_t$ depends on the starting state $s_t$ and the policy $\pi(a \mid s)$. From this state, the agent proceeds through a sequence of states, taking actions and receiving rewards. This sequence can vary each time the agent starts from the same state because the policy $\pi(a_t \mid s_t)$, the state transitions $\Pr(s_{t+1} \mid s_t, a_t)$, and the rewards $\Pr(r_{t+1} \mid s_t, a_t)$ are generally stochastic.

We can evaluate how "good" a state is under a given policy $\pi$ by considering the expected return $v_\pi(s_t)$. This value represents the average return starting from state $s_t$ and following policy $\pi$ thereafter. It is known as the **state-value function**:

$$
v_\pi(s_t) = \mathbb{E}[G_t \mid s_t]
$$

Informally, the state-value function tells us the expected long-term reward if we start in state $s_t$ and follow policy $\pi$. States that are likely to lead to higher rewards in the near future will have higher values, especially when the discount factor $\gamma$ is less than one.

Similarly, the **action-value function** $q_\pi(s_t, a_t)$ is the expected return after taking action $a_t$ in state $s_t$ and then following policy $\pi$:

$$
q_\pi(s_t, a_t) = \mathbb{E}[G_t \mid s_t, a_t]
$$

The action-value function quantifies the expected long-term reward starting from state $s_t$, taking action $a_t$, and subsequently following policy $\pi$. This function is crucial because it connects future rewards to current actions, helping address the temporal credit assignment problem in reinforcement learning.

### Optimal Policy

Our goal is to find a policy that maximizes the expected return. For MDPs (but not necessarily for POMDPs), there exists a deterministic, stationary policy that maximizes the value for every state. The **optimal state-value function** $v^*(s_t)$ is defined as the maximum expected return achievable from state $s_t$:

$$
v^*(s_t) = \max_{\pi} \mathbb{E}[G_t \mid s_t]
$$

Similarly, the **optimal action-value function** $q^*(s_t, a_t)$ is the maximum expected return achievable from state $s_t$ after taking action $a_t$:

$$
q^*(s_t, a_t) = \max_{\pi} \mathbb{E}[G_t \mid s_t, a_t]
$$

If we know the optimal action-value function $q^*(s_t, a_t)$, we can derive the optimal policy by selecting the action that maximizes this function:

$$
\pi^*(a_t \mid s_t) = \arg\max_{a_t} \, q^*(s_t, a_t)
$$

Many reinforcement learning algorithms focus on estimating the action-value function to derive the optimal policy.

### Bellman Equations

While we may not know the exact state-value function $v_\pi(s_t)$ or action-value function $q_\pi(s_t, a_t)$ for a given policy, we can establish relationships between them. The state-value function can be expressed in terms of the action-value function:

$$
v_\pi(s_t) = \sum_{a_t} \pi(a_t \mid s_t) \, q_\pi(s_t, a_t)
$$

This equation indicates that the value of a state is the expected value of the action-values, weighted by the probability of taking each action under policy $\pi$.

Similarly, the action-value function can be defined using the immediate reward and the expected value of the next state:

$$
q_\pi(s_t, a_t) = \mathbb{E} \left[ r_{t+1} + \gamma v_\pi(s_{t+1}) \mid s_t, a_t \right]
$$

Since the next state $s_{t+1}$ is not deterministic, we can expand this to:

$$
q_\pi(s_t, a_t) = \sum_{s_{t+1}} \Pr(s_{t+1} \mid s_t, a_t) \left[ r(s_t, a_t, s_{t+1}) + \gamma v_\pi(s_{t+1}) \right]
$$

Combining these equations leads to the **Bellman equation** for the state-value function:

$$
v_\pi(s_t) = \sum_{a_t} \pi(a_t \mid s_t) \left[ \sum_{s_{t+1}} \Pr(s_{t+1} \mid s_t, a_t) \left( r(s_t, a_t, s_{t+1}) + \gamma v_\pi(s_{t+1}) \right) \right]
$$

Similarly, the Bellman equation for the action-value function is:

$$
q_\pi(s_t, a_t) = \sum_{s_{t+1}} \Pr(s_{t+1} \mid s_t, a_t) \left[ r(s_t, a_t, s_{t+1}) + \gamma \sum_{a_{t+1}} \pi(a_{t+1} \mid s_{t+1}) \, q_\pi(s_{t+1}, a_{t+1}) \right]
$$

These Bellman equations are fundamental to many reinforcement learning methods. They express the idea that the value of a state (or state-action pair) must be consistent with the values of its possible successor states, given the policy and the environment dynamics. Updating the estimated value of one state or action influences others, creating a ripple effect throughout the state space.

## Tabular Reinforcement Learning

Tabular reinforcement learning (RL) algorithms, which operate on finite state and action spaces without function approximation, are generally categorized into model-based and model-free methods.

- **Model-Based Methods**: These methods explicitly utilize the known structure of the Markov Decision Process (MDP), including the transition probabilities $\Pr(s_{t+1} \mid s_t, a_t)$ and the reward function $r(s, a)$, to compute the optimal policy. When these components are known, finding the optimal policy becomes a straightforward optimization problem that can be solved using dynamic programming techniques. If the transition probabilities and reward functions are unknown, they must first be estimated from observed trajectories within the MDP.

- **Model-Free Methods**: These methods do not rely on a model of the MDP and are divided into two main categories:

  1. **Value-Based Methods**: Estimate the optimal state-action value function (Q-function) and derive the policy by selecting the action with the highest value in each state.
  
  2. **Policy-Based Methods**: Directly search for the optimal policy, often using gradient descent techniques, without explicitly estimating value functions or the environment model.

Both categories can further be subdivided based on how they learn from interactions with the environment:

- **Monte Carlo Methods**: These methods rely on simulating multiple episodes or trajectories using a given policy to collect experience, which is then used to improve the policy. They update the policy after observing the returns at the end of episodes.

- **Temporal Difference (TD) Methods**: These methods update the policy incrementally at each time step as the agent interacts with the environment, making updates based on observed transitions without waiting for the end of episodes.

Next, we will briefly discuss dynamic programming methods, Monte Carlo methods for value estimation, and temporal difference methods for value estimation.

### Dynamic Programming

**Dynamic Programming** methods require complete knowledge of the environment's dynamics—that is, the transition probabilities and reward functions are known. This distinguishes them from most reinforcement learning algorithms, which typically learn these quantities indirectly through interaction with the environment.

In dynamic programming, the state values $v(s)$ are initialized arbitrarily (often to zero). Similarly, the policy $\pi(a \mid s)$ is initialized, possibly by assigning a random action to each state. The algorithm then iteratively performs two main steps: policy evaluation and policy improvement.

**Policy Evaluation:** In this step, we update the value function $v(s_t)$ for each state by applying the Bellman expectation equation:

$$
v(s_t) \leftarrow \sum_{a_t} \pi(a_t \mid s_t) \left[ r(s_t, a_t) + \gamma \sum_{s_{t+1}} \Pr(s_{t+1} \mid s_t, a_t) v(s_{t+1}) \right],
$$

where $s_{t+1}$ represents the possible successor states and $\Pr(s_{t+1} \mid s_t, a_t)$ are the state transition probabilities. This update adjusts $v(s_t)$ to be consistent with the estimated values of the successor states, a process known as **bootstrapping**.

**Policy Improvement:** After evaluating the current policy, we update it by choosing the action that maximizes the expected value in each state:

$$
\pi(a_t \mid s_t) \leftarrow \underset{a_t}{\operatorname{argmax}} \left[ r(s_t, a_t) + \gamma \sum_{s_{t+1}} \Pr(s_{t+1} \mid s_t, a_t) v(s_{t+1}) \right].
$$

According to the **policy improvement theorem**, this step is guaranteed to produce a policy that is at least as good as the previous one.

These two steps are repeated alternately until the policy converges to the optimal policy.

There are several variations of dynamic programming methods:

- **Policy Iteration**: The policy evaluation step is performed until the value function converges before performing policy improvement.

- **Value Iteration**: Instead of fully evaluating the policy, we perform only a single sweep of updates before improving the policy. This method combines policy evaluation and improvement into a single step.

- **Asynchronous Dynamic Programming**: Instead of updating all states in each iteration, we update a subset of states in any order. This can be more efficient in certain cases.

### Monte Carlo Methods

Monte Carlo methods differ from dynamic programming in that they do not require prior knowledge of the environment's dynamics. Instead, they learn directly from experience by sampling episodes—sequences of states, actions, and rewards—generated by interacting with the environment. These methods alternate between estimating the action-value function based on the collected experience and updating the policy accordingly.

To estimate the action-value function $q(s, a)$, the agent runs multiple episodes starting from various state-action pairs and following the current policy thereafter. The action-value for a particular state-action pair is calculated as the average of the returns observed after visiting that pair across all episodes. Once the action-values are estimated, the policy is updated by selecting, for each state, the action with the highest estimated value:

$$
\pi(a \mid s) \leftarrow \underset{a}{\operatorname{argmax}} \, q(s, a).
$$

This approach is considered an **on-policy** method because the agent uses the current policy to generate episodes and improve upon it. However, a challenge arises in estimating the values of state-action pairs that are rarely or never visited under the current policy. To ensure all actions are explored, one strategy is to use **exploring starts**, where episodes are initiated from every possible state-action pair, guaranteeing that each is sampled at least once. Unfortunately, this is often impractical in environments with large state or action spaces.

An alternative is to employ an **$\epsilon$-greedy policy**, where the agent selects a random action with probability $\epsilon$, and follows the current optimal action with probability $1 - \epsilon$. This introduces randomness into the action selection process, encouraging exploration of less-visited actions. The parameter $\epsilon$ balances the trade-off between exploration and exploitation. However, when using an on-policy method with an $\epsilon$-greedy policy, the learned policy will be optimized within the family of $\epsilon$-greedy policies, which may not be the absolute optimal policy.

In contrast, **off-policy** methods learn the optimal policy (called the **target policy** $\pi$) using data collected from a different policy (called the **behavior policy** $\pi'$). Typically, the behavior policy is stochastic and encourages exploration (e.g., an $\epsilon$-greedy policy), while the target policy is deterministic and aims for optimal performance. This setup allows the agent to explore the environment thoroughly while still learning an efficient policy.

Some off-policy methods use techniques like **importance sampling** to adjust for the difference between the behavior and target policies when estimating the action-values. Other methods, such as **Q-learning**, directly estimate the optimal action-value function by considering the maximum possible value of the next state, regardless of the action actually taken by the behavior policy.

### Temporal Difference Methods

**Temporal Difference (TD) methods** blend the ideas of dynamic programming and Monte Carlo methods by combining bootstrapping with learning from sampled experience. Unlike Monte Carlo methods, which wait until the end of an episode to update value estimates, TD methods update estimates incrementally at each time step as the agent interacts with the environment.

One example of a TD method is **SARSA** (State-Action-Reward-State-Action), an on-policy algorithm that updates the action-value function using the following rule:

$$
q(s_t, a_t) \leftarrow q(s_t, a_t) + \alpha \left[ r(s_t, a_t) + \gamma \, q(s_{t+1}, a_{t+1}) - q(s_t, a_t) \right],
$$

where $\alpha > 0$ is the learning rate. The term inside the brackets is known as the **TD error**, which represents the difference between the current estimate $q(s_t, a_t)$ and a bootstrap estimate $r(s_t, a_t) + \gamma \, q(s_{t+1}, a_{t+1})$ based on the next state and action.

In contrast, **Q-learning** is an off-policy TD method that updates the action-value function using:

$$
q(s_t, a_t) \leftarrow q(s_t, a_t) + \alpha \left[ r(s_t, a_t) + \gamma \, \max_{a} q(s_{t+1}, a) - q(s_t, a_t) \right].
$$

Here, the update uses the maximum estimated value of the next state over all possible actions, rather than the value of the action actually taken. This means that the learning process is independent of the agent's behavior policy $\pi'$, which can be exploratory, while the update targets the optimal policy.

In both SARSA and Q-learning, the policy is typically updated by selecting the action with the highest estimated action-value in each state. Under certain conditions, these update rules are guaranteed to converge to the optimal action-value function, provided that all state-action pairs are visited infinitely often and the learning rate decreases appropriately over time.

## Fitted Q-Learning

Traditional tabular Monte Carlo and Temporal Difference (TD) algorithms require exhaustive traversal of the entire Markov Decision Process (MDP) to update action values. This approach is only feasible when the state-action space is relatively small. In most practical scenarios, however, the state-action space is immense; for example, the game of chess has over $10^{40}$ possible legal states despite its seemingly constrained environment.

In **fitted Q-learning**, we replace the discrete action value representation $q[s_t, a_t]$ with a function approximator $q(\mathbf{s}_t, a_t; \boldsymbol{\phi})$, where $\mathbf{s}_t$ is a feature vector representing the state, and $\boldsymbol{\phi}$ are the parameters of the model. We define a least squares loss function based on the temporal difference error:

$$
L(\boldsymbol{\phi}) = \left( r(\mathbf{s}_t, a_t) + \gamma \cdot \max_a q(\mathbf{s}_{t+1}, a; \boldsymbol{\phi}) - q(\mathbf{s}_t, a_t; \boldsymbol{\phi}) \right)^2,
$$

which leads to the following parameter update rule:

$$
\boldsymbol{\phi} \leftarrow \boldsymbol{\phi} + \alpha \left( r(\mathbf{s}_t, a_t) + \gamma \cdot \max_a q(\mathbf{s}_{t+1}, a; \boldsymbol{\phi}) - q(\mathbf{s}_t, a_t; \boldsymbol{\phi}) \right) \frac{\partial q(\mathbf{s}_t, a_t; \boldsymbol{\phi})}{\partial \boldsymbol{\phi}}.
$$

In this formulation, adjusting the parameters $\boldsymbol{\phi}$ affects both the target value $r(\mathbf{s}_t, a_t) + \gamma \cdot \max_{a} q(\mathbf{s}_{t+1}, a; \boldsymbol{\phi})$ and the predicted value $q(\mathbf{s}_t, a_t; \boldsymbol{\phi})$, since both depend on $\boldsymbol{\phi}$. This coupling can lead to instability and convergence issues, as changes in the parameters can cause the target values to shift unpredictably.

### Deep Q-Networks for Playing Atari Games

Deep neural networks are well-suited for handling high-dimensional input spaces, making them a natural choice for approximating the action-value function in fitted Q-learning. While it's possible to input both state and action into the network, it's more efficient in practice to input only the state and have the network output the values for all possible actions simultaneously.

The **Deep Q-Network (DQN)** architecture was a significant advancement in reinforcement learning, demonstrating the ability to learn to play Atari 2600 games directly from raw pixel input. The raw game frames are $210 \times 160$ pixel images with potentially 128 color values per pixel. To preprocess the input, the images are converted to grayscale, resized to $84 \times 84$ pixels, and normalized. However, a single frame doesn't capture motion information like object velocities. To address this, the network stacks the last four frames to form the state representation $\mathbf{s}_t$.

The network architecture processes these frames through three convolutional layers, capturing spatial and temporal information, followed by fully connected layers that output the estimated value for each possible action.

Several key techniques were employed to stabilize and improve training:

- **Reward Clipping**: Game scores can vary dramatically across different games. To normalize the reward signal, the game score changes were clipped to a range of $[-1, 1]$. This standardization allows for consistent training parameters across various games.

- **Experience Replay**: Instead of updating the network with sequential samples, experiences $\langle \mathbf{s}_t, a_t, r_{t+1}, \mathbf{s}_{t+1} \rangle$ are stored in a replay buffer. Random batches are sampled from this buffer for training, breaking the temporal correlations between samples and improving data efficiency by reusing past experiences.

To address convergence issues associated with changing target values, DQN uses a separate target network with parameters $\boldsymbol{\phi}^-$. The target network's parameters are periodically updated to match the primary network's parameters but remain fixed between updates. The update rule becomes:

$$
\boldsymbol{\phi} \leftarrow \boldsymbol{\phi} + \alpha \left( r(\mathbf{s}_t, a_t) + \gamma \cdot \max_a q(\mathbf{s}_{t+1}, a; \boldsymbol{\phi}^-) - q(\mathbf{s}_t, a_t; \boldsymbol{\phi}) \right) \frac{\partial q(\mathbf{s}_t, a_t; \boldsymbol{\phi})}{\partial \boldsymbol{\phi}}.
$$

This approach stabilizes training by keeping the target values relatively constant over short periods, preventing the network from chasing a moving target.

Using these methods along with an $\epsilon$-greedy exploration strategy, DQNs achieved performance comparable to professional human players on a suite of 49 Atari games, using the same network architecture (trained separately for each game). Training was computationally intensive, requiring around 38 days of gameplay experience per game. While the DQN surpassed human performance in some games, it struggled with others like **Montezuma's Revenge**, which involves sparse rewards and complex exploration due to its multiple, visually distinct screens.

### Double Q-Learning and Double Deep Q-Networks

A limitation of standard Q-learning is that the maximization step in the update rule:

$$
q[s_t, a_t] \leftarrow q[s_t, a_t] + \alpha \left( r[s_t, a_t] + \gamma \cdot \max_a q[s_{t+1}, a] - q[s_t, a_t] \right)
$$

can introduce an overestimation bias in the action-value estimates. This bias occurs because the same samples are used to select and evaluate actions, which can lead to the overestimation of action values, especially in noisy environments or when using function approximation.

**Double Q-Learning** mitigates this issue by decoupling the action selection from the evaluation. It maintains two separate value functions, $q_1$ and $q_2$, each with its own parameters. The update rules are:

$$
\begin{aligned}
q_1[s_t, a_t] &\leftarrow q_1[s_t, a_t] + \alpha \left( r[s_t, a_t] + \gamma \cdot q_2[s_{t+1}, \operatorname{argmax}_a q_1[s_{t+1}, a]] - q_1[s_t, a_t] \right), \\
q_2[s_t, a_t] &\leftarrow q_2[s_t, a_t] + \alpha \left( r[s_t, a_t] + \gamma \cdot q_1[s_{t+1}, \operatorname{argmax}_a q_2[s_{t+1}, a]] - q_2[s_t, a_t] \right).
\end{aligned}
$$

In this approach, one value function selects the best action (via the $\operatorname{argmax}$), while the other evaluates the action's value. By alternating the roles of $q_1$ and $q_2$, the method reduces the overestimation bias inherent in standard Q-learning.

Extending this concept to deep neural networks leads to **Double Deep Q-Networks (Double DQNs)**. Two networks with parameters $\boldsymbol{\phi}_1$ and $\boldsymbol{\phi}_2$ are used. The update rules are:

$$
\begin{aligned}
\boldsymbol{\phi}_1 &\leftarrow \boldsymbol{\phi}_1 + \alpha \left( r(\mathbf{s}_t, a_t) + \gamma \cdot q(\mathbf{s}_{t+1}, \operatorname{argmax}_a q(\mathbf{s}_{t+1}, a; \boldsymbol{\phi}_1); \boldsymbol{\phi}_2) - q(\mathbf{s}_t, a_t; \boldsymbol{\phi}_1) \right) \frac{\partial q(\mathbf{s}_t, a_t; \boldsymbol{\phi}_1)}{\partial \boldsymbol{\phi}_1}, \\
\boldsymbol{\phi}_2 &\leftarrow \boldsymbol{\phi}_2 + \alpha \left( r(\mathbf{s}_t, a_t) + \gamma \cdot q(\mathbf{s}_{t+1}, \operatorname{argmax}_a q(\mathbf{s}_{t+1}, a; \boldsymbol{\phi}_2); \boldsymbol{\phi}_1) - q(\mathbf{s}_t, a_t; \boldsymbol{\phi}_2) \right) \frac{\partial q(\mathbf{s}_t, a_t; \boldsymbol{\phi}_2)}{\partial \boldsymbol{\phi}_2}.
\end{aligned}
$$

Here, each network uses the other network's parameters to evaluate the selected action, effectively decoupling action selection from evaluation. This technique reduces overestimation bias, leading to more accurate value estimates and improved learning stability.

By integrating Double DQNs into the training process, agents can achieve better performance, particularly in environments where the standard Q-learning overestimation bias adversely affects learning.

## Policy Gradient Methods

While Q-learning methods estimate action values to derive policies, **policy gradient methods** directly optimize a stochastic policy $\pi(a_t \mid \mathbf{s}_t; \boldsymbol{\theta})$. This policy is a parameterized function with trainable parameters $\boldsymbol{\theta}$ that maps a state $\mathbf{s}_t$ to a probability distribution over actions $a_t$. Even though there exists an optimal deterministic policy in Markov Decision Processes (MDPs), using a stochastic policy has several advantages:

1. **Enhanced Exploration**: A stochastic policy inherently promotes exploration by allowing the agent to occasionally choose suboptimal actions, which can lead to the discovery of better strategies.

2. **Smooth Optimization**: Stochastic policies enable the use of gradient-based optimization methods since the output probabilities change smoothly with respect to the parameters $\boldsymbol{\theta}$, even when dealing with discrete rewards. This is analogous to maximum likelihood estimation in classification tasks.

3. **Handling Partial Observability**: In many real-world scenarios, agents have incomplete knowledge of the environment's state. A stochastic policy allows an agent to take different actions in states that appear similar but are actually different, helping to resolve ambiguities over time.

### Derivation of the Gradient Update

Consider a trajectory $\tau = [\mathbf{s}_1, a_1, \mathbf{s}_2, a_2, \ldots, \mathbf{s}_T, a_T]$ representing a sequence of states and actions in an MDP. The probability of this trajectory under the policy $\pi$ is given by:

$$
\operatorname{Pr}(\tau \mid \boldsymbol{\theta}) = \operatorname{Pr}(\mathbf{s}_1) \prod_{t=1}^T \pi(a_t \mid \mathbf{s}_t; \boldsymbol{\theta}) \operatorname{Pr}(\mathbf{s}_{t+1} \mid \mathbf{s}_t, a_t).
$$

Our goal is to find the parameters $\boldsymbol{\theta}$ that maximize the expected return over all possible trajectories:

$$
\boldsymbol{\theta} = \underset{\boldsymbol{\theta}}{\operatorname{argmax}} \left[ \mathbb{E}_{\tau}[r(\tau)] \right] = \underset{\boldsymbol{\theta}}{\operatorname{argmax}} \left[ \int \operatorname{Pr}(\tau \mid \boldsymbol{\theta}) r(\tau) \, d\tau \right],
$$

where $r(\tau)$ is the total reward accumulated along trajectory $\tau$.

To perform gradient ascent on the expected return, we update the parameters:

$$
\boldsymbol{\theta} \leftarrow \boldsymbol{\theta} + \alpha \frac{\partial}{\partial \boldsymbol{\theta}} \mathbb{E}_{\tau}[r(\tau)] = \boldsymbol{\theta} + \alpha \int \frac{\partial \operatorname{Pr}(\tau \mid \boldsymbol{\theta})}{\partial \boldsymbol{\theta}} r(\tau) \, d\tau,
$$

where $\alpha$ is the learning rate.

We can approximate this integral using samples from the current policy. By multiplying and dividing the integrand by $\operatorname{Pr}(\tau \mid \boldsymbol{\theta})$, we obtain:

$$
\begin{aligned}
\boldsymbol{\theta} & \leftarrow \boldsymbol{\theta} + \alpha \int \operatorname{Pr}(\tau \mid \boldsymbol{\theta}) \left( \frac{1}{\operatorname{Pr}(\tau \mid \boldsymbol{\theta})} \frac{\partial \operatorname{Pr}(\tau \mid \boldsymbol{\theta})}{\partial \boldsymbol{\theta}} r(\tau) \right) d\tau \\
& \approx \boldsymbol{\theta} + \alpha \frac{1}{I} \sum_{i=1}^I \left( \frac{1}{\operatorname{Pr}(\tau_i \mid \boldsymbol{\theta})} \frac{\partial \operatorname{Pr}(\tau_i \mid \boldsymbol{\theta})}{\partial \boldsymbol{\theta}} r(\tau_i) \right),
\end{aligned}
$$

where $I$ is the number of sampled trajectories.

Using the identity $\frac{\partial}{\partial z} \log f(z) = \frac{1}{f(z)} \frac{\partial f(z)}{\partial z}$, we simplify the update rule:

$$
\boldsymbol{\theta} \leftarrow \boldsymbol{\theta} + \alpha \frac{1}{I} \sum_{i=1}^I \frac{\partial \log \operatorname{Pr}(\tau_i \mid \boldsymbol{\theta})}{\partial \boldsymbol{\theta}} r(\tau_i).
$$

Since only the policy $\pi(a_t \mid \mathbf{s}_t; \boldsymbol{\theta})$ depends on $\boldsymbol{\theta}$, we have:

$$
\frac{\partial \log \operatorname{Pr}(\tau \mid \boldsymbol{\theta})}{\partial \boldsymbol{\theta}} = \sum_{t=1}^T \frac{\partial \log \pi(a_t \mid \mathbf{s}_t; \boldsymbol{\theta})}{\partial \boldsymbol{\theta}}.
$$

Thus, the parameter update becomes:

$$
\boldsymbol{\theta} \leftarrow \boldsymbol{\theta} + \alpha \frac{1}{I} \sum_{i=1}^I \sum_{t=1}^T \frac{\partial \log \pi(a_{it} \mid \mathbf{s}_{it}; \boldsymbol{\theta})}{\partial \boldsymbol{\theta}} r(\tau_i).
$$

Because rewards obtained before time $t$ do not influence the action at time $t$, we can focus on future rewards:

$$
r(\tau_i) = \sum_{k=1}^T r_{ik} = \sum_{k=1}^{t-1} r_{ik} + \sum_{k=t}^T r_{ik}.
$$

Therefore, we can rewrite the update as:

$$
\boldsymbol{\theta} \leftarrow \boldsymbol{\theta} + \alpha \frac{1}{I} \sum_{i=1}^I \sum_{t=1}^T \frac{\partial \log \pi(a_{it} \mid \mathbf{s}_{it}; \boldsymbol{\theta})}{\partial \boldsymbol{\theta}} \sum_{k=t}^T r_{ik}.
$$

### REINFORCE Algorithm

The **REINFORCE** algorithm is a foundational policy gradient method that incorporates discounting of future rewards. It operates by generating episodes $\tau_i = [\mathbf{s}_{i1}, a_{i1}, r_{i2}, \mathbf{s}_{i2}, a_{i2}, r_{i3}, \ldots, \mathbf{s}_{iT}, a_{iT}, r_{iT+1}]$ using the current policy $\pi(a \mid \mathbf{s}; \boldsymbol{\theta})$. For discrete actions, the policy can be parameterized by a neural network producing probabilities over actions via a softmax layer.

For each episode $i$, we perform the following steps:

1. **Calculate Discounted Returns**: For each time step $t$, compute the discounted return from that point forward:

   $$
   G_{it} = \sum_{k=t}^{T} \gamma^{k - t} r_{i, k+1},
   $$

   where $\gamma \in [0, 1)$ is the discount factor.

2. **Update Parameters**: Adjust the policy parameters using:

   $$
   \boldsymbol{\theta} \leftarrow \boldsymbol{\theta} + \alpha \sum_{t=1}^T \gamma^{t} \frac{\partial \log \pi(a_{it} \mid \mathbf{s}_{it}; \boldsymbol{\theta})}{\partial \boldsymbol{\theta}} G_{it},
   $$

   where $\alpha$ is the learning rate. The term $\gamma^{t}$ accounts for the discounting relative to the start of the episode.

### Baselines

Policy gradient methods can suffer from high variance, requiring many episodes to obtain stable gradient estimates. To reduce this variance without introducing bias, we subtract a baseline value $b$ from the returns:

$$
\boldsymbol{\theta} \leftarrow \boldsymbol{\theta} + \alpha \frac{1}{I} \sum_{i=1}^I \sum_{t=1}^T \frac{\partial \log \pi(a_{it} \mid \mathbf{s}_{it}; \boldsymbol{\theta})}{\partial \boldsymbol{\theta}} \left( G_{it} - b \right).
$$

As long as the baseline $b$ does not depend on the action $a_{it}$, its expected contribution to the gradient is zero:

$$
\mathbb{E}_{\tau} \left[ \sum_{t=1}^T \frac{\partial \log \pi(a_{it} \mid \mathbf{s}_{it}; \boldsymbol{\theta})}{\partial \boldsymbol{\theta}} \cdot b \right] = 0.
$$

This ensures that the baseline reduces variance without biasing the gradient estimate—a technique known as the method of control variates.

To choose an optimal baseline $b$, we can minimize the variance of the gradient estimator. The optimal baseline is given by:

$$
b = \frac{\sum_{i=1}^I \sum_{t=1}^T \left\| \frac{\partial \log \pi(a_{it} \mid \mathbf{s}_{it}; \boldsymbol{\theta})}{\partial \boldsymbol{\theta}} \right\|^2 G_{it}}{\sum_{i=1}^I \sum_{t=1}^T \left\| \frac{\partial \log \pi(a_{it} \mid \mathbf{s}_{it}; \boldsymbol{\theta})}{\partial \boldsymbol{\theta}} \right\|^2}.
$$

In practice, this is often approximated by the average return over all sampled trajectories:

$$
b \approx \frac{1}{I} \sum_{i=1}^I G_{i1}.
$$

Subtracting this baseline helps to reduce variance caused by fluctuations in overall returns unrelated to the actions taken.

### State-Dependent Baselines

An even more effective strategy is to use a baseline that depends on the current state, $b(\mathbf{s}_{it})$:

$$
\boldsymbol{\theta} \leftarrow \boldsymbol{\theta} + \alpha \frac{1}{I} \sum_{i=1}^I \sum_{t=1}^T \frac{\partial \log \pi(a_{it} \mid \mathbf{s}_{it}; \boldsymbol{\theta})}{\partial \boldsymbol{\theta}} \left( G_{it} - b(\mathbf{s}_{it}) \right).
$$

This approach accounts for the fact that some states inherently lead to higher returns, regardless of the action taken, further reducing variance.

A logical choice for the state-dependent baseline is the **state-value function** $v(\mathbf{s}_{it})$, which estimates the expected return from state $\mathbf{s}_{it}$. The difference $G_{it} - v(\mathbf{s}_{it})$ is known as the **advantage estimate**.

Since the true state-value function may be unknown, we can approximate it using a parameterized function $v(\mathbf{s}; \boldsymbol{\phi})$, such as a neural network. We train this value function to minimize the difference between predicted and actual returns using a mean squared error loss:

$$
L(\boldsymbol{\phi}) = \sum_{i=1}^I \sum_{t=1}^T \left( v(\mathbf{s}_{it}; \boldsymbol{\phi}) - G_{it} \right)^2.
$$

By integrating the state-dependent baseline into the policy gradient update, we achieve a more stable and efficient learning process.

## Actor-Critic Methods

Actor-critic methods are a class of temporal difference (TD) policy gradient algorithms that combine elements of both value-based and policy-based reinforcement learning. Unlike Monte Carlo methods like REINFORCE—which require complete episodes to update the policy parameters—actor-critic algorithms can update the policy at each time step.

In the TD approach, we do not have access to the exact future rewards $ G_t = \sum_{k=t}^T \gamma^{k - t} r_k $ along the trajectory. Instead, actor-critic algorithms approximate the expected return using the observed immediate reward and the estimated value of the next state:

$$
G_t \approx r_t + \gamma \, v(\mathbf{s}_{t+1}; \boldsymbol{\phi})
$$

Here, $ v(\mathbf{s}_{t+1}; \boldsymbol{\phi}) $ is the estimated value of the next state $ \mathbf{s}_{t+1} $, parameterized by $ \boldsymbol{\phi} $. Substituting this approximation into the policy gradient update yields:

$$
\boldsymbol{\theta} \leftarrow \boldsymbol{\theta} + \alpha \sum_{t=1}^T \frac{\partial \log \pi(a_t \mid \mathbf{s}_t; \boldsymbol{\theta})}{\partial \boldsymbol{\theta}} \left[ r_t + \gamma \, v(\mathbf{s}_{t+1}; \boldsymbol{\phi}) - v(\mathbf{s}_t; \boldsymbol{\phi}) \right]
$$

The term in brackets is known as the **TD error** $ \delta_t $:

$$
\delta_t = r_t + \gamma \, v(\mathbf{s}_{t+1}; \boldsymbol{\phi}) - v(\mathbf{s}_t; \boldsymbol{\phi})
$$

Simultaneously, we update the value function parameters $ \boldsymbol{\phi} $ by minimizing the following loss function:

$$
L(\boldsymbol{\phi}) = \left[ r_t + \gamma \, v(\mathbf{s}_{t+1}; \boldsymbol{\phi}) - v(\mathbf{s}_t; \boldsymbol{\phi}) \right]^2
$$

In this framework:

- The **actor** is the policy network $ \pi(a_t \mid \mathbf{s}_t; \boldsymbol{\theta}) $, which outputs the probability distribution over actions given the current state.
- The **critic** is the value network $ v(\mathbf{s}_t; \boldsymbol{\phi}) $, which estimates the expected return from state $ \mathbf{s}_t $.

Often, both the actor and the critic are combined into a single neural network that shares some layers but has separate output layers for the policy and value estimates.

While actor-critic methods are capable of updating policy parameters at each time step, in practice, it's common to collect a batch of experiences over several time steps before performing updates. This approach helps stabilize training by reducing variance in gradient estimates and improving learning efficiency.

## Offline Reinforcement Learning

Interaction with the environment is fundamental to reinforcement learning. However, in certain scenarios, deploying an inexperienced agent to explore various actions is impractical or unsafe. For instance, erratic behavior could be dangerous in contexts like autonomous vehicle navigation, or data collection might be costly and time-consuming, as seen in financial trading.

In such cases, historical data collected from human agents can be leveraged. **Offline Reinforcement Learning** (also known as batch RL) aims to learn policies that maximize rewards in future episodes by analyzing past sequences $\mathbf{s}_1, a_1, r_2, \mathbf{s}_2, a_2, r_3, \ldots$, without interacting with the environment. This approach differs from imitation learning, which (i) lacks access to reward information and (ii) focuses on replicating the behavior of the historical agent rather than improving upon it.

While there are offline RL methods based on Q-learning and policy gradients, this paradigm opens up new possibilities. Specifically, we can frame the problem as a sequence learning task, where the goal is to predict the next action given the history of states, actions, and rewards. The **Decision Transformer** utilizes a transformer decoder architecture to make these predictions.

However, predicting actions based on future rewards is challenging with standard $\mathbf{s}, a, r$ sequences. To address this, the Decision Transformer replaces the immediate reward $r_t$ with the **return-to-go** $R_{t:T} = \sum_{t'=t}^T r_{t'}$, which represents the sum of future rewards from time $t$ to $T$. The rest of the model mirrors a typical transformer decoder: states, actions, and returns-to-go are converted into fixed-size embeddings through learned mappings. For example, in Atari games, state embeddings might be generated via a convolutional neural network. The embeddings for actions and returns-to-go can be learned similarly to word embeddings. The transformer is trained using masked self-attention and positional embeddings.

While this formulation works well during training, it poses a challenge during inference because the returns-to-go are unknown in advance. This issue can be mitigated by specifying a desired total return at the initial step and decrementing it as rewards are collected. For instance, in an Atari game, the desired total return might be set to the winning score.

Decision Transformers can also be fine-tuned using online experience, allowing them to learn and adapt over time. They have the advantage of bypassing much of the traditional reinforcement learning infrastructure and its associated instability, instead relying on standard supervised learning techniques. Transformers can learn from vast amounts of data and capture long-range temporal dependencies, making the temporal credit assignment problem more manageable. This approach represents an intriguing new direction in reinforcement learning.